Some lat/longs are incorrect. They are given the default location of City Hall.

This affects Census block group, Census data, and imputed zipcode for NaNs.

Since address is still given, we could impute via Google Maps API, but there are 42495 such rows.

For the sake of time, I will regettably drop them and do a quick EDA on what they are, in case dropping them might bias our results.

In [1]:
from __future__ import division
import pandas as pd

In [5]:
from zipcode_mapping import zipcode_mapping

In [2]:
df = pd.read_pickle('../data/data_w_transformed_census_and_removed_invalid_rows_and_cols_and_fixed_zips.pkl')
df.shape

(786867, 36)

In [6]:
df['neighborhood_from_zip'] = df.zipcode.map(zipcode_mapping)

## EDA

In [7]:
df[df.neighborhood.notnull()][df.neighborhood_from_zip != df.neighborhood] \
    [df.LATITUDE == 42.3594].shape

/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


(42495, 37)

In [8]:
df_incorrect_lat_longs = df[df.neighborhood.notnull()][df.neighborhood_from_zip != df.neighborhood] \
    [df.LATITUDE == 42.3594].copy()

/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


In [10]:
df_incorrect_lat_longs[['TYPE', 'LOCATION_STREET_NAME', 'neighborhood', 'neighborhood_from_zip']].head()

,TYPE,LOCATION_STREET_NAME,neighborhood,neighborhood_from_zip
905649,Misc. Snow Complaint,INTERSECTION M St & Marine Rd,South Boston / South Boston Waterfront,Dorchester
905648,Request for Pothole Repair,INTERSECTION Lyndeboro St & Middlesex St,Charlestown,Dorchester
905442,Parking Meter Repairs,INTERSECTION Lyndeboro St & Middlesex St,Charlestown,Dorchester
905524,Sign Repair,INTERSECTION Williams St & Amory St,Jamaica Plain,Dorchester
905503,Transportation General Request,INTERSECTION Itasca St & Cummins Hwy,Greater Mattapan,Dorchester


In [12]:
df_incorrect_lat_longs \
    [['TYPE', 'neighborhood']].groupby('TYPE').count().sort_values('neighborhood', ascending=False).head()  

,neighborhood
TYPE,
Parking Enforcement,5921
Requests for Street Cleaning,4270
Traffic Signal Inspection,3751
Sign Repair,3057
Request for Pothole Repair,2681


## Dropping them

In [13]:
df.shape, df_incorrect_lat_longs.shape

((786867, 37), (42495, 37))

In [17]:
offending_case_enquiry_ids = df_incorrect_lat_longs['CASE_ENQUIRY_ID']

In [18]:
offending_case_enquiry_ids.shape

(42495,)

In [19]:
new_df = df[~df.CASE_ENQUIRY_ID.isin(offending_case_enquiry_ids)]
new_df.shape

(744372, 37)

In [20]:
744372 + 42495

786867

## Saving

In [21]:
new_df.to_pickle('../data/data_w_transformed_census_and_removed_invalid_rows_and_cols_and_fixed_zips_and_descs_and_dropped_latlongs.pkl')